In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from psaw import PushshiftAPI

In [3]:
# Initialize API
api = PushshiftAPI()

# Getting all submissions with more than a certain upvote score

In [2]:
# Initialize API
api = PushshiftAPI()

# Set up generator to make API request.
api_request_generator = api.search_submissions(subreddit='news', score = ">2000")

# Make the request and collect results into a pd.DataFrame
news_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

# Shape (n_results, n_features)
news_submissions.shape

(14878, 98)

In [19]:
# Available features
news_submissions.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'edited', 'full_link', 'gilded', 'gildings',
       'id', 'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'thumbnail', 'title', 'total_awards_received',
       'treatment_ta

In [17]:
# Transform to datetime
news_submissions['date'] = pd.to_datetime(jokes_submissions['created_utc'], utc = True, unit = 's')
news_submissions[['date','title', 'score', 'num_comments', 'selftext']].sample(20)

,date,title,score,num_comments,selftext
747,2019-07-24 03:28:46+00:00,"My best friend called me and said ""An evil wiz...",12514,117,I drove all the way to his house just to find ...
822,2019-07-10 09:43:25+00:00,I went to the liquor store on my bicycle and b...,6395,93,\n\n...'cause I fell 7 times on the way home...
161,2020-04-15 14:42:22+00:00,Husband doing crossword with his wife,2528,132,"\n\nHusband: Emphatic no, five letters.\n\nWi..."
1312,2019-04-18 20:13:24+00:00,The girl with no arms and legs laying by the pool,9047,238,There’s a girl with no arms and legs laying by...
5600,2016-12-21 09:12:29+00:00,Break ups are the worst in China...,3047,180,You see her face everywhere.
2864,2018-04-05 16:00:16+00:00,I asked my girlfriend to describe me in 5 words.,34538,537,"She said I'm mature, I'm moral, I'm pure, I'm ..."
5254,2017-02-18 22:10:34+00:00,Two blind pilots enter a plane,4247,100,They have sunglasses and white sticks. As the ...
2569,2018-05-19 15:06:42+00:00,Don't ever underestimate a Scottish police off...,5240,144,A London lawyer runs a stop sign and gets pull...
3232,2018-02-12 05:04:36+00:00,Afternoon Sex,36740,663,"The only way to pull off a Sunday afternoon ""q..."
711,2019-07-30 11:44:48+00:00,I once dated a girl who had a twin.,22951,356,People kept asking me how I could tell them ap...


# Getting submission based on search keyword

Searching comments is done in the same way, but using api.search_comments instead of api.search_submissions

In [28]:
# Set up generator to make API request.
api_request_generator = api.search_submissions(q='(jobs | employment)', score = '>2000')

q_jobs_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])

q_jobs_submissions['date'] = pd.to_datetime(q_jobs_submissions['created_utc'], utc=True, unit='s')

q_jobs_submissions.shape

(4645, 102)

In [29]:
q_jobs_submissions[['date','title', 'score', 'num_comments', 'selftext']].sample(20)

,date,title,score,num_comments,selftext
115,2020-07-21 18:38:33+00:00,Aita for calling out my sister after she shame...,12490,1036,I’m 28 my sister is 26. She got married three ...
2041,2018-03-05 16:21:16+00:00,TIL Before he was a famous musician Johnny Cas...,32066,499,
3571,2016-10-09 18:06:11+00:00,My boyfriend and I started a business out of h...,3612,2867,Hi Reddit! I’m Monique. Two years ago I was wo...
3687,2016-07-24 12:55:42+00:00,EMSK the most popular resumes are either chron...,2039,54,[deleted]
3202,2017-01-26 12:45:34+00:00,Keystone pipeline will create just 35 permanen...,31452,3040,
577,2019-08-15 07:08:55+00:00,Video Game Developer Insight on EA's Relations...,2581,460,I've been a video game developer for near thre...
1973,2018-03-23 15:14:04+00:00,I work as a companion to a man who has Down sy...,3045,251,His Individual Support Plan allows for no alon...
687,2019-07-19 10:47:41+00:00,Fine! But you're stealing their jobs!,3068,163,I love Target. I'm also a stress eater who hap...
3446,2016-11-30 00:42:18+00:00,You are only a Temp!,3459,253,So in my time out of tech support from the lig...
3683,2016-07-26 04:35:07+00:00,Ken M On Six-Figure Jobs,3944,163,


In [30]:
max(q_jobs_submissions['date'])

Timestamp('2021-06-14 02:29:15+0000', tz='UTC')

In [ ]:
# Collect submissions/comments within a certain period

In [3]:
start_time = int(datetime(2020, 10, 23).timestamp())
end_time = int(datetime(2020, 10, 26).timestamp())

api_request_generator = api.search_submissions(q='(jobs | employment)',
                                              after = start_time,
                                              before = end_time)

q_jobs_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])
q_jobs_submissions.shape

(6205, 99)

In [5]:
start_time = int(datetime(2020, 10, 23).timestamp())
end_time = int(datetime(2020, 10, 26).timestamp())

api_request_generator2 = api.search_submissions(q='(jobs OR employment)',
                                              after = start_time,
                                              before = end_time)

q_jobs_submissions2 = pd.DataFrame([submission.d_ for submission in api_request_generator2])
q_jobs_submissions2.shape

(65, 71)

In [11]:
q_jobs_submissions['date'] = pd.to_datetime(q_jobs_submissions['created_utc'], utc=True, unit='s')
q_jobs_submissions[['date','title', 'score', 'num_comments', 'selftext']].sample(20)

,date,title,score,num_comments,selftext
1630,2020-10-25 03:22:31+00:00,I’m addicted to caffeine pills,1,16,I probably take 1000mg-1400mg is caffeine a da...
4054,2020-10-23 21:50:56+00:00,Question about gaps after a foundation program...,1,4,"Hello, I wanna ask about a gap in trade.\n\nI'..."
5805,2020-10-23 03:14:45+00:00,HOW THIS WORKS...,1,0,\n* Website of the course: https://LinuxUpskil...
5259,2020-10-23 10:29:21+00:00,Revamp your problems by the help of real black...,1,0,&amp;#x200B;\n\n[ ](https://preview.redd.it/t...
5724,2020-10-23 03:54:07+00:00,Scrapping together a network,1,5,So I got a bunch of gear that our shop has acc...
2594,2020-10-24 16:20:41+00:00,Our new home. ♥️ It’s about 400 years old. We ...,1,34,
1979,2020-10-24 22:44:32+00:00,Can you get the full maternity EI amount over ...,1,8,Let's say maternity EI pays you 500 a week for...
1034,2020-10-25 12:58:31+00:00,"Amateur &amp; POV doggy, anal, blow jobs, cum ...",1,2,
5072,2020-10-23 12:37:24+00:00,What exactly does the 'Check Access' button in...,1,3,Hello all\n\nWe are having trouble with device...
3810,2020-10-24 00:26:20+00:00,The job market seriously stinks,1,8,I enlisted in the Air Force at the start of th...


In [ ]:
q_jobs_submissions['selftext', 'subreddit'][5805]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 611
Traceback (most recent call last):
  File "/Users/munchausend/opt/anaconda3/envs/redditScrapper/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: ('selftext', 'subreddit')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/munchausend/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.u

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/munchausend/opt/anaconda3/envs/redditScrapper/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: ('selftext', 'subreddit')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/munchausend/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-026950f2ef79>", line 1, in <module>
    q

In [7]:
q_jobs_submissions2['date'] = pd.to_datetime(q_jobs_submissions2['created_utc'], utc=True, unit='s')
q_jobs_submissions2[['date','title', 'score', 'num_comments', 'selftext']].sample(20)

,date,title,score,num_comments,selftext
27,2020-10-24 18:22:02+00:00,Do I have to list ALL past employment on appli...,1,12,I am 29 and applying for law school for the fi...
21,2020-10-25 01:52:09+00:00,Why are so many people relying on a stimulus?,1,202,This is not meant to be demeaning or insulting...
45,2020-10-23 19:46:58+00:00,I never thought I could hate someone so much.,1,2,My belief that being friends with an ex is ver...
41,2020-10-23 23:36:18+00:00,35 [M4F] I want to turn you into white trash,1,0,"Have you ever wanted to just ""let go""? To sto..."
16,2020-10-25 08:08:50+00:00,I'm a public policy analyst that has dedicated...,1,9,"Hey peeps,\n\nMy name is Tudor! By trade, I a..."
4,2020-10-25 20:25:33+00:00,"Was super excited about finally getting a job,...",1,0,I don’t even know how to start with this. Apol...
10,2020-10-25 15:59:03+00:00,Hi there. I'm thinking of applying to the Robe...,1,3,Hi there. I'm thinking of applying to the Robe...
43,2020-10-23 21:55:57+00:00,"Moving to Winnipeg from Edmonton, what is the ...",1,21,I currently work for the Government of Alberta...
44,2020-10-23 20:18:48+00:00,prop 22,1,13,Disclaimer: I'm sorry if this does not belong ...
48,2020-10-23 16:45:32+00:00,[HIRING] Research Data Analyst at University o...,1,0,"University of California, Riverside is looking..."


In [13]:
q_jobs_submissions2['selftext'][45]

'My belief that being friends with an ex is very rare, and even in those rarities there are many attempting to be friends and haven’t quite realized the ship they’re trying to salvage is already sunk. \n\nI happen to be in one of these situations. At this point the only options is ghosting, as bad as it sounds, it is the only healthy choice. I’m not the greatest person in the world I may even share some of the Narc traits my ex has, it could be one of the biggest reasons I feel I hate her. But it’s not the only reason by a long shot.\n\nShe’s reached out and I have reached out when I know I shouldn’t have. Anytime we connect now it’s usually me who starts an argument because shes doing and saying the same things that added to our inviability. She starts complaining about someone making her feel a certain way, there’s something for her to complain about constantly, and she still talks about her ex, comparing me to him, and I’m sitting there like wait wtf I don’t have to listen to any of

In [48]:
start_time = int(datetime(2020, 10, 23).timestamp())
end_time = int(datetime(2020, 10, 26).timestamp())

api_request_generator = api.search_comments(q='(jobs | employment)',
                                              after = start_time,
                                              before = end_time)

q_jobs_comments = pd.DataFrame([submission.d_ for submission in api_request_generator])
q_jobs_comments['date'] = pd.to_datetime(q_jobs_comments['created_utc'], utc=True, unit='s')
q_jobs_comments.shape

(35706, 39)

In [51]:
q_jobs_comments.columns

Index(['all_awardings', 'associated_award', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
       'body', 'collapsed_because_crowd_control', 'comment_type',
       'created_utc', 'gildings', 'id', 'is_submitter', 'link_id', 'locked',
       'no_follow', 'parent_id', 'permalink', 'retrieved_on', 'score',
       'send_replies', 'stickied', 'subreddit', 'subreddit_id',
       'top_awarded_type', 'total_awards_received', 'treatment_tags',
       'created', 'distinguished', 'edited', 'author_cakeday', 'date'],
      dtype='object')

In [ ]:
q_jobs_comments[['date', 'score', 'body', 'parent_id']].sample(20)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 612
Traceback (most recent call last):
  File "/Users/munchausend/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-cd530e1fd048>", line 1, in <module>
    q_jobs_comments[['date', 'score', 'body', 'parent_id']].sample(20)
NameError: name 'q_jobs_comments' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/munchausend/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/munchausend/.local/lib/python3.7/site-packages/IPytho

## Submission & comments

In [4]:
start_time = int(datetime(2020, 10, 23).timestamp())
end_time = int(datetime(2020, 10, 26).timestamp())

api_request_generator = api.search_submissions(q='(jobs | employment)',
                                              after = start_time,
                                              before = end_time)

q_jobs_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])
q_jobs_submissions.shape

(6205, 99)

In [5]:
start_time = int(datetime(2020, 10, 23).timestamp())
end_time = int(datetime(2020, 10, 26).timestamp())

api_request_generator = api.search_comments(q='(jobs | employment)',
                                              after = start_time,
                                              before = end_time)

q_jobs_comments = pd.DataFrame([submission.d_ for submission in api_request_generator])
q_jobs_comments['date'] = pd.to_datetime(q_jobs_comments['created_utc'], utc=True, unit='s')
q_jobs_comments.shape

(35706, 40)

In [6]:
q_jobs_comments.columns

Index(['all_awardings', 'associated_award', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_template_id',
       'author_flair_text', 'author_flair_text_color', 'author_flair_type',
       'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders',
       'body', 'collapsed_because_crowd_control', 'comment_type',
       'created_utc', 'gildings', 'id', 'is_submitter', 'link_id', 'locked',
       'no_follow', 'parent_id', 'permalink', 'retrieved_on', 'score',
       'send_replies', 'stickied', 'subreddit', 'subreddit_id',
       'top_awarded_type', 'total_awards_received', 'treatment_tags',
       'created', 'distinguished', 'edited', 'author_cakeday', 'date'],
      dtype='object')

In [33]:
submission_ids = q_jobs_submissions['id']
comment_parent_ids = q_jobs_comments['parent_id']
comment_link_ids = q_jobs_comments['link_id']

In [35]:
len(set(comment_parent_ids) - set(comment_link_ids))

20723

In [36]:
len(set(comment_link_ids) - set(comment_parent_ids))

8419

In [39]:
comment_parent_ids.sample(20)

21325    t1_g9tuw82
4516     t1_ga2hj5f
25752    t1_g9rfglv
23539    t1_g9snrtz
17030     t3_jgwv0u
13691     t3_jhecjb
9554     t1_g9zsrj7
7050      t3_jhr6l0
22168     t3_jgya81
8502     t1_g9z2jql
19892    t1_g9u9a0w
83        t3_jh9dhm
28912    t1_g9r8odr
24547     t3_jgspm0
30489     t3_jgjrof
9085     t1_ga01hrn
22397    t1_g9s50eo
20898     t3_jgzwf7
33499    t1_g9kpy3v
15320    t1_g9wvxar
Name: parent_id, dtype: object

In [40]:
comment_link_ids.sample(20)

17511    t3_jh2ddd
11328    t3_jh5y1y
15283    t3_jhb1qu
4506     t3_jh7rkj
23812    t3_jgq2v8
18034    t3_jh66pe
14889    t3_jh6qga
13144    t3_jh1sif
7043     t3_jhppl3
25579    t3_jgse4d
21581    t3_jgz9bo
12743    t3_jhbkz4
32135    t3_jgf6bl
5421     t3_jh7u4w
21485    t3_jgej3a
21607    t3_jgz4zn
13745    t3_jh7yav
16103    t3_jh6yw9
3793     t3_jhx5my
28909    t3_jgky0e
Name: link_id, dtype: object

Will use link_id to actually link comments to submissions.
>link_id is the submission. parent_id is the parent of the comment. It could be another comment or it could be the submission if the comment is a top level comment

In [46]:
comment_link_ids[1]

't3_ji19pv'

In [49]:
comment_link_ids = [x.split('_')[1] for x in comment_link_ids]

In [55]:
len(set((comment_link_ids)) - set(submission_ids))

17327

In [56]:
len(set(comment_link_ids))

18543

## Filtering

In [42]:
api_request_generator = api.search_submissions(q='(jobs | employment)',
                                              after = start_time,
                                              before = end_time, 
                                              filter=['url','author', 'title', 'subreddit'],
                                              limit = 10000)

q_jobs_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])
q_jobs_submissions.shape

/Users/munchausend/opt/anaconda3/envs/redditScrapper/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/Users/munchausend/opt/anaconda3/envs/redditScrapper/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


(6205, 6)

In [43]:
q_jobs_submissions.columns

Index(['author', 'created_utc', 'subreddit', 'title', 'url', 'created'], dtype='object')

The files that are returned can be filtered by adding ```filter = ['field1','field2']``` in the api call.